In [48]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy import linalg
import algoritmo_ncm as ncm
import sys
import os

In [49]:

class HiddenPrints:
    """Classe que desabilita prints dentro de funções"""
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [50]:
def gera_matrizes(n):
    #Devolve uma matriz inicial e a que queremos achar a ncm
    x = np.random.randn(n, n)
    g = np.random.randn(n, n)

    #A matriz a qual queremos encontrar a ncm mais proxima precisa ser simétrica 
    g = g@g.T
    return x,g 

In [51]:
def teste_solução(x,g,max_iter,tol):
#Função testa se a solução é valida
    x, iter = ncm.sm_ncm(x,g,max_iter,tol)
    if np.any(np.linalg.eigvals(x)) < -tol:
        print("não resolve")
    else: 
        print("resolve")

In [52]:
def media_iter(n,laps,max_iter,tol):
    lista_iter = []
    for i in range(laps):
        x,g  = gera_matrizes(n)
        with HiddenPrints():
            x, iter = ncm.sm_ncm(x,g,max_iter,tol)
        lista_iter.append(iter)
    media = np.mean(np.array(lista_iter))
    return lista_iter, media


In [64]:
# Gerar as matrizes fora do bloco timeit
n = 30
laps = 100
lista_g = []
lista_x0 = []
for i in range(laps):
    x0, g = gera_matrizes(n)
    lista_g.append(g)
    lista_x0.append(x0)

def mede_tempo(lista_x0,lista_g,laps,max_iter,tol):
    """Uma função que mede o tempo sem gerar as matrizes nem os vetores,o resultado é o tempo
    de 100 iterações"""
    with HiddenPrints():
        for i in range(laps):
            x, iter = ncm.sm_ncm(lista_x0[i],lista_g[i],max_iter,0.00001)
    return x

%timeit mede_tempo(lista_x0,lista_g,100,2000,0.00001)

13 s ± 64.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy import linalg
from func_auxiliares_ncm import *


def diferencial_f(x, g):
    """Função responsável por fazer uma iteração do algoritmo"""
    df = np.linalg.inv(np.diag(np.diag(derivada_clarke(x))))@(e(x) - np.diag(derivada_clarke(x)@sem_diag_g(g)))
    return df


def nearest_correlation_matrix(x_input, g_input, iter = 1000, tol = 10**(-6)):
    """Algoritmo de Nearest Correlation Matrix"""
    iteracao = 1
    X_anterior = x_input
    X_atual = np.diag(diferencial_f(X_anterior, g_input)) + sem_diag_g(g_input)

    while i < iter:
        i = i + 1
        print("iteracao", i - 1)
        print(derivada_clarke(X_anterior) @ X_atual)
        X_anterior = X_atual
        X_atual = np.diag(diferencial_f(X_anterior, g_input)) + sem_diag_g(g_input)

        if np.linalg.norm(np.diag(derivada_clarke(X_atual) @ X_atual) - e(X_atual)) < tol:
            break      
        
    return derivada_clarke(X_anterior) @ X_atual, iteracao

In [2]:
import numpy as np

def projecao_S(R):
    # Implement the projection onto the semidefinite subset
    # Placeholder implementation, replace with actual projection logic
    eigvals, eigvecs = np.linalg.eigh(R)
    eigvals[eigvals < 0] = 0
    return eigvecs @ np.diag(eigvals) @ eigvecs.T

def projecao_U(Y):
    # Implementa a projeção no subconjunto de diagonal unitária
    # Implementação de espaço reservado, substitua pela lógica de projeção real
    np.fill_diagonal(Y, 1)
    return Y

def projecoes_alternadas(G, max_iteracoes=1000):

    delta_S = np.zeros_like(G)
    X = G.copy()

    for k in range(1, max_iteracoes + 1):

        R = X - delta_S
        
        # Projeta no subconjunto semidefinido
        Y = projecao_S(R)
        

        delta_S = Y - R
        print("Iteração", k,X)
        # Projeta no subconjunto de diagonal unitária
        X = projecao_U(Y)
    
    return X

# Exemplo de uso


In [ ]:
#crie uma matriz G 4x4 e teste com as funções projeções alternadas e sm_ncm
G = np.random.randn(4, 4)
G = G @ G.T
X = projecoes_alternadas(G)
print(X)
X, iter = nearest_correlation_matrix(X, G, 1000, 1e-6)
print(X)




In [3]:
new_matrix = np.array([
    [2, -1, 0, 0],
    [-1, 2, -1, 0],
    [0, -1, 2, -1],
    [0, 0, -1, 2]
])

In [4]:
# Teste da função projeções alternadas com new_matrix
X_new = projecoes_alternadas(new_matrix)
print("Resultado da projeção alternada:")
print(X_new)

# Teste da função sm_ncm com new_matrix
X_sm_ncm, iter_sm_ncm = nearest_correlation_matrix(X_new, new_matrix, 1000, 1e-6)
print("Resultado do sm_ncm:")
print(X_sm_ncm)
print("Número de iterações:", iter_sm_ncm)

Iteração 1 [[ 2 -1  0  0]
 [-1  2 -1  0]
 [ 0 -1  2 -1]
 [ 0  0 -1  2]]
Iteração 2 [[ 1.00000000e+00 -1.00000000e+00  0.00000000e+00 -2.22044605e-16]
 [-1.00000000e+00  1.00000000e+00 -1.00000000e+00 -6.10622664e-16]
 [-1.11022302e-16 -1.00000000e+00  1.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00 -6.66133815e-16 -1.00000000e+00  1.00000000e+00]]
Iteração 3 [[ 1.        -0.8618034  0.1381966  0.0854102]
 [-0.8618034  1.        -0.7763932  0.1381966]
 [ 0.1381966 -0.7763932  1.        -0.8618034]
 [ 0.0854102  0.1381966 -0.8618034  1.       ]]
Iteração 4 [[ 1.         -0.82506789  0.17493211  0.10172275]
 [-0.82506789  1.         -0.69917013  0.17493211]
 [ 0.17493211 -0.69917013  1.         -0.82506789]
 [ 0.10172275  0.17493211 -0.82506789  1.        ]]
Iteração 5 [[ 1.         -0.81405097  0.18594903  0.10532286]
 [-0.81405097  1.         -0.67170433  0.18594903]
 [ 0.18594903 -0.67170433  1.         -0.81405097]
 [ 0.10532286  0.18594903 -0.81405097  1.        ]]
Iteração 6 [[ 1.

NameError: name 'G_chapeu' is not defined